In [58]:
import datetime as dt

start_time = dt.datetime.now(tz=dt.UTC)
MAX_SECONDS = 60 * 10 # aim for 10 minutes run time

In [59]:
# Comment these lines out when developing locally
# ! pip install ephem 'flaml[automl]' # TODO uncomment
# %cd /kaggle/working # TODO uncomment

In [60]:
import math
from pathlib import Path

import ephem
import kagglehub
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import requests
from kagglehub.config import DEFAULT_CACHE_FOLDER

In [61]:
class SunPosition:
    def __init__(self, *, latitude: float, longitude: float) -> None:
        self.latitude = latitude
        self.longitude = longitude
        self._observer = self._create_ephem_observer()
        self._sun = ephem.Sun()

    def _create_ephem_observer(self) -> ephem.Observer:
        observer = ephem.Observer()
        observer.lat = str(self.latitude)
        observer.lon = str(self.longitude)
        return observer

    def altitude(self, *, timestamp_utc: dt.datetime) -> float:
        self._observer.date = timestamp_utc
        self._sun.compute(self._observer)
        return self._sun.alt

In [62]:
CACHE_DIR = Path(DEFAULT_CACHE_FOLDER) / "competitions" / "hill-of-towie-wind-turbine-power-prediction"


def load_training_dataset(*, force_download: bool = False, just_for_year:int|None=None) -> pl.LazyFrame:
    file_path = kagglehub.competition_download(
        handle="hill-of-towie-wind-turbine-power-prediction",
        path="training_dataset.parquet",
        force_download=force_download,
    )
    if just_for_year is None:
        return pl.scan_parquet(Path(file_path))
    return pl.scan_parquet(Path(file_path)).filter(pl.col("TimeStamp_StartFormat").dt.year() == just_for_year)


def load_submission_dataset(*, force_download: bool = False) -> pl.LazyFrame:
    file_path = kagglehub.competition_download(
        handle="hill-of-towie-wind-turbine-power-prediction",
        path="submission_dataset.parquet",
        force_download=force_download,
    )
    return pl.scan_parquet(Path(file_path))

def filter_is_valid(X: pl.DataFrame, y: pl.Series) -> tuple[pl.DataFrame, pl.Series]:
    y = y.filter(X.select("is_valid").to_series())
    X = X.filter(pl.col("is_valid"))
    return X, y

In [63]:
# just load 2019 because that is more representative of best case wind-up analysis where pre and post are at most 1 year
df_train = load_training_dataset(just_for_year=2019).collect()
df_train.head(5)

TimeStamp_StartFormat,wtc_AcWindSp_mean;1,wtc_AcWindSp_mean;2,wtc_AcWindSp_mean;3,wtc_AcWindSp_mean;4,wtc_AcWindSp_mean;5,wtc_AcWindSp_mean;7,wtc_AcWindSp_min;1,wtc_AcWindSp_min;2,wtc_AcWindSp_min;3,wtc_AcWindSp_min;4,wtc_AcWindSp_min;5,wtc_AcWindSp_min;7,wtc_AcWindSp_max;1,wtc_AcWindSp_max;2,wtc_AcWindSp_max;3,wtc_AcWindSp_max;4,wtc_AcWindSp_max;5,wtc_AcWindSp_max;7,wtc_AcWindSp_stddev;1,wtc_AcWindSp_stddev;2,wtc_AcWindSp_stddev;3,wtc_AcWindSp_stddev;4,wtc_AcWindSp_stddev;5,wtc_AcWindSp_stddev;7,wtc_ScYawPos_mean;1,wtc_ScYawPos_mean;2,wtc_ScYawPos_mean;3,wtc_ScYawPos_mean;4,wtc_ScYawPos_mean;5,wtc_ScYawPos_mean;7,wtc_ScYawPos_min;1,wtc_ScYawPos_min;2,wtc_ScYawPos_min;3,wtc_ScYawPos_min;4,wtc_ScYawPos_min;5,wtc_ScYawPos_min;7,…,wtc_ActPower_max;2,wtc_ActPower_max;3,wtc_ActPower_max;4,wtc_ActPower_max;5,wtc_ActPower_max;7,wtc_ActPower_stddev;1,wtc_ActPower_stddev;2,wtc_ActPower_stddev;3,wtc_ActPower_stddev;4,wtc_ActPower_stddev;5,wtc_ActPower_stddev;7,wtc_AmbieTmp_mean;1,wtc_AmbieTmp_mean;2,wtc_AmbieTmp_mean;3,wtc_AmbieTmp_mean;4,wtc_AmbieTmp_mean;5,wtc_AmbieTmp_mean;7,ShutdownDuration;1,ShutdownDuration;2,ShutdownDuration;3,ShutdownDuration;4,ShutdownDuration;5,ShutdownDuration;7,ERA5_temperature_2m,ERA5_relative_humidity_2m,ERA5_dew_point_2m,ERA5_precipitation,ERA5_surface_pressure,ERA5_cloud_cover,ERA5_wind_speed_10m,ERA5_wind_speed_100m,ERA5_wind_direction_10m,ERA5_wind_direction_100m,ERA5_wind_gusts_10m,id,is_valid,target
"datetime[μs, UTC]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i16,i16,i16,i16,i16,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i32,bool,f64
2019-01-01 00:00:00 UTC,12.27432,13.16435,11.24778,11.08846,10.63383,13.0237,4.98,7.57,5.28,5.49,4.65,7.78,17.790001,21.66,16.32,18.84,18.0,20.790001,2.353884,2.328966,2.005096,2.536227,2.682431,2.399789,-54.786541,-90.360458,-74.553673,-70.138687,-93.22419,-57.71698,-55.299999,-94.699997,-77.599998,-71.0,-96.5,-59.299999,…,2390.0,2358.0,2358.0,2349.0,2352.0,375.827087,159.997101,403.552887,460.781403,612.429077,297.708893,4.291883,5.0,5.0,4.212133,5.0,4.626184,0,0,0,0,0,0,6.642,70.984154,1.752,0.0,994.82843,70.0,8.448077,13.345411,286.504425,287.891968,18.4,-52560,true,2005.649048
2019-01-01 00:10:00 UTC,10.87804,12.9349,10.46269,9.208033,10.01653,10.79133,6.42,7.78,5.49,4.49,4.65,6.22,15.8,19.16,16.530001,15.17,15.48,16.950001,1.870514,1.893832,2.004121,2.052478,2.116802,2.282843,-56.252041,-91.302856,-77.594261,-69.643448,-93.010674,-55.443501,-58.700001,-92.400002,-80.5,-71.0,-94.099998,-62.200001,…,2355.0,2342.0,2348.0,2343.0,2352.0,415.182098,203.421402,388.686005,493.755707,579.818481,480.481812,4.129384,5.0,5.0,4.129483,5.0,4.255417,0,0,0,0,0,0,11.292,73.076088,6.642,0.0,990.559143,100.0,8.723532,13.800363,244.903778,246.96492,17.6,-52559,true,1789.771973
2019-01-01 00:20:00 UTC,10.57354,9.532747,12.13545,11.79736,10.21164,12.68395,4.65,4.98,6.84,4.81,5.59,7.15,17.059999,15.48,17.27,16.950001,15.8,17.690001,2.043976,1.990858,2.118668,2.224131,1.962167,1.925842,-44.838291,-85.389122,-65.529373,-63.350052,-86.091339,-51.238529,-56.099998,-91.0,-76.0,-69.900002,-93.099998,-52.5,…,2371.0,2349.0,2373.0,2348.0,2363.0,501.897003,538.811584,413.073914,447.610596,541.481079,328.893799,4.001917,5.0,5.0,4.0499,4.939867,4.131166,0,0,0,0,0,0,9.392,74.005676,4.992,0.0,994.148438,100.0,7.300684,11.847363,246.595291,248.198532,13.7,-52558,true,1654.629028
2019-01-01 00:30:00 UTC,13.00446,11.11443,12.33127,12.81789,11.34538,13.63712,8.19,6.42,6.74,8.3,6.84,7.88,18.530001,16.639999,18.209999,17.48,15.69,19.889999,2.0071,1.945408,1.845071,1.657609,1.745914,2.062722,-43.624008,-78.366547,-61.45805,-61.885479,-82.292229,-49.95879,-44.5,-82.800003,-64.199997,-63.900002,-85.199997,-51.599998,…,2353.0,2352.0,2350.0,2353.0,2354.0,238.3564,431.991486,194.204605,196.636093,385.329712,270.309204,4.0258,5.0,5.0,4.009

In [64]:
df_train.tail(5)

TimeStamp_StartFormat,wtc_AcWindSp_mean;1,wtc_AcWindSp_mean;2,wtc_AcWindSp_mean;3,wtc_AcWindSp_mean;4,wtc_AcWindSp_mean;5,wtc_AcWindSp_mean;7,wtc_AcWindSp_min;1,wtc_AcWindSp_min;2,wtc_AcWindSp_min;3,wtc_AcWindSp_min;4,wtc_AcWindSp_min;5,wtc_AcWindSp_min;7,wtc_AcWindSp_max;1,wtc_AcWindSp_max;2,wtc_AcWindSp_max;3,wtc_AcWindSp_max;4,wtc_AcWindSp_max;5,wtc_AcWindSp_max;7,wtc_AcWindSp_stddev;1,wtc_AcWindSp_stddev;2,wtc_AcWindSp_stddev;3,wtc_AcWindSp_stddev;4,wtc_AcWindSp_stddev;5,wtc_AcWindSp_stddev;7,wtc_ScYawPos_mean;1,wtc_ScYawPos_mean;2,wtc_ScYawPos_mean;3,wtc_ScYawPos_mean;4,wtc_ScYawPos_mean;5,wtc_ScYawPos_mean;7,wtc_ScYawPos_min;1,wtc_ScYawPos_min;2,wtc_ScYawPos_min;3,wtc_ScYawPos_min;4,wtc_ScYawPos_min;5,wtc_ScYawPos_min;7,…,wtc_ActPower_max;2,wtc_ActPower_max;3,wtc_ActPower_max;4,wtc_ActPower_max;5,wtc_ActPower_max;7,wtc_ActPower_stddev;1,wtc_ActPower_stddev;2,wtc_ActPower_stddev;3,wtc_ActPower_stddev;4,wtc_ActPower_stddev;5,wtc_ActPower_stddev;7,wtc_AmbieTmp_mean;1,wtc_AmbieTmp_mean;2,wtc_AmbieTmp_mean;3,wtc_AmbieTmp_mean;4,wtc_AmbieTmp_mean;5,wtc_AmbieTmp_mean;7,ShutdownDuration;1,ShutdownDuration;2,ShutdownDuration;3,ShutdownDuration;4,ShutdownDuration;5,ShutdownDuration;7,ERA5_temperature_2m,ERA5_relative_humidity_2m,ERA5_dew_point_2m,ERA5_precipitation,ERA5_surface_pressure,ERA5_cloud_cover,ERA5_wind_speed_10m,ERA5_wind_speed_100m,ERA5_wind_direction_10m,ERA5_wind_direction_100m,ERA5_wind_gusts_10m,id,is_valid,target
"datetime[μs, UTC]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i16,i16,i16,i16,i16,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i32,bool,f64
2019-12-31 23:10:00 UTC,8.897216,6.674914,8.790455,9.309954,6.749103,9.335896,4.81,3.26,4.4,5.07,3.59,5.59,12.69,10.07,12.45,13.17,9.75,13.52,1.453008,1.169652,1.494632,1.24644,1.121678,1.443997,-96.119614,-126.805603,-117.9804,-112.195999,-136.195602,-101.531799,-100.699997,-131.5,-121.5,-114.800003,-138.0,-105.900002,…,662.0,1592.0,1676.0,693.0,1658.0,184.697006,75.007767,290.05719,192.467102,112.776604,299.1586,3.516783,3.986933,3.992567,3.62495,3.49205,3.162083,0,0,0,0,0,0,1.942,90.765717,0.592,0.0,1001.884033,69.0,3.088689,6.236986,209.054504,221.099411,5.6,-5,true,964.380127
2019-12-31 23:20:00 UTC,9.303802,7.50387,8.951791,9.97668,8.194743,8.700228,6.32,3.59,4.98,6.94,4.57,4.65,12.21,11.5,12.69,13.64,11.26,13.05,1.038562,1.186081,1.556765,1.226175,1.112155,1.936404,-97.002197,-128.177399,-118.5597,-113.142998,-136.962708,-104.914497,-103.0,-134.100006,-121.400002,-115.199997,-140.600006,-106.0,…,1064.0,1581.0,1884.0,1126.0,1650.0,177.164505,129.734695,367.965393,315.084015,153.462906,411.925598,4.467517,4.278967,4.643017,4.41965,4.003917,3.422133,0,0,0,0,0,0,2.842,63.106548,-3.488,0.0,996.454956,95.0,4.326662,8.902246,213.690094,218.157272,8.5,-4,true,1106.759033
2019-12-31 23:30:00 UTC,8.292965,7.242282,7.387715,8.898215,7.778623,7.802809,4.98,4.98,4.49,5.49,5.18,5.28,10.79,9.54,9.85,11.86,10.55,9.75,0.972607,0.881621,0.918921,0.909908,0.89253,0.779341,-96.173378,-124.966599,-116.936699,-113.882103,-133.844696,-102.936096,-103.0,-134.100006,-121.199997,-117.400002,-140.600006,-107.0,…,628.0,673.0,1258.0,761.0,817.0,76.277786,61.76931,71.848351,117.3293,94.306686,58.581848,4.618834,4.787467,3.607067,4.939967,4.704933,2.973367,0,0,0,0,0,0,2.942,61.226002,-3.798,0.0,996.174988,55.0,4.701064,9.276314,218.088821,221.941223,8.8,-3,true,687.353088
2019-12-31 23:40:00 UTC,9.617872,9.914719,8.692904,10.19371,10.44945,8.951912,6.63,5.9,5.9,7.15,6.11,6.22,12.81,14.24,11.5,12.81,14.0,11.26,0.957572,1.496119,1.032007,0.999657,1.459871,0.746953,-83.940231,-113.025002,-106.912804,-102.196404,-121.0886,-94.316544,-92.199997,-121.099998,-112.699997,-110.900002,-134.5,-97.300003,…,1829.0,1221.0,1689.0,2067.0,1025.0,195.377304,362.835999,158.093002,212.542206,361.223389,63.843021,3.230533,4.2476,3.0,3.47136

In [65]:
# basic definitions of X and y train
X_train = df_train.select(pl.exclude("target"))
y_train = df_train.select("target").to_series()


In [66]:
# remove invalid rows
X_train, y_train = filter_is_valid(X_train, y_train)

In [67]:
# preprocess X
def preprocess_x(X):
    cols_to_exclude=["id","is_valid","TimeStamp_StartFormat",*[x for x in X.columns if x.split(";")[-1]=="1"]]
    return X.select(pl.exclude(cols_to_exclude))
X_train = preprocess_x(X_train)
print(X_train.columns)

['wtc_AcWindSp_mean;2', 'wtc_AcWindSp_mean;3', 'wtc_AcWindSp_mean;4', 'wtc_AcWindSp_mean;5', 'wtc_AcWindSp_mean;7', 'wtc_AcWindSp_min;2', 'wtc_AcWindSp_min;3', 'wtc_AcWindSp_min;4', 'wtc_AcWindSp_min;5', 'wtc_AcWindSp_min;7', 'wtc_AcWindSp_max;2', 'wtc_AcWindSp_max;3', 'wtc_AcWindSp_max;4', 'wtc_AcWindSp_max;5', 'wtc_AcWindSp_max;7', 'wtc_AcWindSp_stddev;2', 'wtc_AcWindSp_stddev;3', 'wtc_AcWindSp_stddev;4', 'wtc_AcWindSp_stddev;5', 'wtc_AcWindSp_stddev;7', 'wtc_ScYawPos_mean;2', 'wtc_ScYawPos_mean;3', 'wtc_ScYawPos_mean;4', 'wtc_ScYawPos_mean;5', 'wtc_ScYawPos_mean;7', 'wtc_ScYawPos_min;2', 'wtc_ScYawPos_min;3', 'wtc_ScYawPos_min;4', 'wtc_ScYawPos_min;5', 'wtc_ScYawPos_min;7', 'wtc_ScYawPos_max;2', 'wtc_ScYawPos_max;3', 'wtc_ScYawPos_max;4', 'wtc_ScYawPos_max;5', 'wtc_ScYawPos_max;7', 'wtc_ScYawPos_stddev;2', 'wtc_ScYawPos_stddev;3', 'wtc_ScYawPos_stddev;4', 'wtc_ScYawPos_stddev;5', 'wtc_ScYawPos_stddev;7', 'wtc_NacelPos_mean;2', 'wtc_NacelPos_mean;3', 'wtc_NacelPos_mean;4', 'wtc_Nacel

In [68]:
from sklearn.ensemble import HistGradientBoostingRegressor
common_params = {"l2_regularization": 1.0, "random_state": 0}
model = HistGradientBoostingRegressor(**common_params)
model.fit(X_train, y_train)

HistGradientBoostingRegressor(l2_regularization=1.0, random_state=0)

In [69]:
X_test = load_submission_dataset().collect()
df_id = X_test.select("id")
X_test = preprocess_x(X_test)
y_test = pl.Series(values=model.predict(X_test.to_pandas())).clip(lower_bound=0)
submission = df_id.with_columns(prediction=y_test)

In [70]:
# checking the columns are the expected ones
assert submission.columns == ["id", "prediction"], f'Expected columns ["id", "prediction"], found: {submission.columns}'

# checking no nulls in the data
assert submission.select(pl.col("id").is_null().sum()).item() == 0, "There are null values in the 'id' column"
assert submission.select(pl.col("id").is_nan().sum()).item() == 0, "There are nan values in the 'id' column"
assert submission.select(pl.col("prediction").is_null().sum()).item() == 0, (
    "There are null values in the 'prediction' column"
)
assert submission.select(pl.col("prediction").is_nan().sum()).item() == 0, (
    "There are nan values in the 'prediction' column"
)

# checking the row ids are unique and within expected range
duplicated_ids = submission.select("id").is_duplicated()
assert not duplicated_ids.any(), (
    f"There are duplicated ids: {submission.select('id').filter(duplicated_ids).to_series().unique()}"
)
invalid_ids = set(submission.select("id").unique().to_series().to_list()) - set(range(52704))
assert not invalid_ids, f"The following row IDs are not within the expected ones: {invalid_ids}"

print("Submission file is valid and ready for submission.")

submission.write_csv("submission.csv")

Submission file is valid and ready for submission.
